Imports

In [13]:
import torch
from torchvision import datasets, transforms, models

from torch.utils.tensorboard import SummaryWriter

Set up neural network model
Use pretrained Resnet 101 (https://storage.googleapis.com/openimages/web/extras.html)

In [14]:
net =  models.resnet101(pretrained=False)
net.load_state_dict(torch.load('weights.pth'))
net.eval()
image_size = 224  # 224 for ENet-B0, 600 for ENet-B7

load images from folder './data'

In [15]:
data_dir = 'data'
data_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomResizedCrop(image_size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # mu, sigma for each channel
])

test_data = datasets.ImageFolder(data_dir, data_transforms)
dataloader = torch.utils.data.DataLoader(test_data)

Run and print predictions

In [24]:
preds = []
for input, _ in dataloader:
    output = net(input)
    is_stair = (output[0][stair_id] >= 0.5)
    # TODO: calculate 90% quantile
    preds.append(is_stair)  # true if stairs recognized, else false
print(preds)

[tensor(True), tensor(False), tensor(True), tensor(False), tensor(False), tensor(True), tensor(False), tensor(True)]


Visualize in TensorBoard

In [ ]:
dataiter = iter(dataloader)
images, labels = next(dataiter)
with SummaryWriter('summary') as writer:
    writer.add_graph(net, images)

## To-Dos
- convert TF models to PyTorch using ONNX converter
- test, finetune classification